In [ ]:
import pandas as pd; pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
import numpy as np
from datetime import datetime
import warnings; warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import HTML, display
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
import ipywidgets as widgets
import arrow

#import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine, 
    FSMOperator, filterFSM, FSMPlot_Start, 
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, 
    disp_result, alarms_pareto, warnings_pareto, states_lines,
    detect_edge_right, detect_edge_left, get_cycle_data)
    
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();

In [ ]:
# def sfun(x):
#     return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
# fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = mp.search_installed_fleet_by_contains_name("Pforzheim")
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

In [ ]:
motor = fleet.iloc[0]
modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
#success = [True]
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
e=Engine.from_fleet(mp,motor)
#fsm = FSMOperator(e, p_from=e['Commissioning Date'], p_to=datetime.now(), successtime=300)
fsm = FSMOperator(e, p_from="2021-10-01", successtime=300)
#fsm = FSMOperator(e, p_to='2022-02-01', successtime=600)
#fsm = FSMOperator(e, p_from='2022-01-26', p_to='2022-02-01', successtime=600)

#fsm.restore()
fsm.run1(enforce=True) # run Finite State Machine
fsm.store()

In [ ]:
rdf = fsm.starts
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
display(HTML(pd.DataFrame.from_dict(e.dash, orient='index').T.to_html(escape=False, index=False)))
#rda[startstopFSM.run2filter_content].round(2)
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == True].shape[0]}, Failed: {rdf[rdf['success'] == False].shape[0]} => {rdf[rdf['success'] == True].shape[0]/rdf.shape[0]*100.0:3.1f}%")

In [ ]:
rda[startstopFSM.run2filter_content].round(2).fillna('')

In [ ]:
fsm.results['starts'][0];

In [ ]:
#Execute FSM Run 2
# fsm.restore()
# rda = fsm.run2(rda, silent=False)
# fsm.store()

In [ ]:
#rdb = rda[(rda['mode'].isin(['AUTO','MAN'])) & ((rda['count_alarms'] > 0) | (rda['count_warnings'] > 0))]
#rdb = rda[rda['synchronize'] < 15.0]
rdb = rda
#rdb[startstopFSM.run2filter_content].sort_values(by=["synchronize","loadramp"], ascending=[True,True]).round(2)

In [ ]:
vec = ['startpreparation','speedup','idle','synchronize','loadramp','maxload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout']
display(_=rdb[vec].hist(bins=30,figsize=(20,20)))
#ax_list[0][2].set_xlim((0,10))
#display(rdb[vec].hist(bins=30,figsize=(20,20)))
display(rdb[vec].describe().round(2))

In [ ]:
rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success'])].copy()
rde['datetime'] = pd.to_datetime(rde['starttime'])
dfigsize = (18,10)
dset = [
    {'col':['cumstarttime'],'ylim':(-400,900), 'color':'darkblue'},
    {'col':['synchronize'],'ylim':(-20,300)},
    {'col':['startpreparation'],'ylim':(-600,300)},
    {'col':['hochlauf'],'ylim':(-100,200), 'color':'orange'},
    {'col':['loadramp'],'ylim':(-150,900), 'color':'red'},
    {'col':['ramprate'],'ylim':(-3,3)},
    {'col':['no'],'ylim':(-100,10000),'color':'black'},
    {'col':['maxload'],'ylim':(500,5000) }
]
ftitle = f"{fsm._e}"
fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
bokeh_show(fig)
del(fig)

In [ ]:
from dowork import plot_now
vset = ['Various_Values_SpeedAct','Power_PowerAct','Various_Values_PosTurboBypassA1','Various_SetValues_SetWasteGate','Various_Values_PressBoostP2A','Power_SetPower','Exhaust_TempCylMax','Exhaust_TempCylMin','Hyd_PressCrankCase','Hyd_PressOilDif','TecJet_Lambda1','Hyd_PressOil','Hyd_TempOil']
dset = [
    {'col':['Power_SetPower'], 'ylim':(0,15000), 'color':'dodgerblue'},
    {'col':['Power_PowerAct'], 'ylim':(0,15000), 'color':'red'},
    {'col':['Various_Values_PosTurboBypassA1'], '_ylim':(0,15000), 'color':'black'},
    {'col':['Various_SetValues_SetWasteGate'], '_ylim':(0,15000), 'color':'MediumVioletRed'},
    {'col':['Various_Values_PressBoostP2A'], '_ylim':(0,15000), 'color':'MediumVioletRed'},
    {'col':['power_diff'], '_ylim':(0,5000), 'color':'purple'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue'},
    {'col':['Exhaust_TempCylMax'],'ylim': [200, 700], 'color':'darkred'},
    {'col':['Exhaust_TempCylMin'],'ylim': [200, 700], 'color':'salmon'},
    {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange'},
    {'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black'},
    {'col':['TecJet_Lambda1'],'_ylim': [0, 2], 'color':'green'},
    {'col':['Hyd_PressOil'],'_ylim': [0, 10], 'color':'brown'},
    {'col':['Hyd_TempOil'],'_ylim': [0, 110], 'color':'#2171b5'}
]


In [ ]:
rdb[startstopFSM.run2filter_content].sort_values(by=["synchronize","loadramp"], ascending=[True,True]).round(2).fillna('');
rdb[startstopFSM.run2filter_content]

In [ ]:
def f(x=0):
    global v; v = x; startversuch = rdb.iloc[x]
    ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
    display(HTML(ftitle));
    disp_result(startversuch)
interact(f, x = IntSlider(0, 0, rdb.shape[0]-1 , 1, layout=widgets.Layout(width='70%')));

In [ ]:
inerestingstarts_Pforzheim_M1 = []
inerestingstarts_Pforzheim_M4 = [41,42,54,67,68,69,70]
startversuch = rdb.iloc[v]; 
data = get_cycle_data2(fsm, startversuch, max_length=None, min_length=None, cycletime=1, silent=False, p_data=vset)

data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
fig = plot_now(fsm, data, startversuch,vset, dset, dfigsize=(26,14))
#fsm.starts.iloc[v].to_dict()
print(data.loc[data['Exhaust_TempCylMax'].idxmax()])
bokeh_show(fig)

In [ ]:
fsm.starts.iloc[v].to_dict();

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)

In [ ]:
fsm._e.BMEP

In [ ]:
fsm._e._calc_BMEP(11318,1000)

In [ ]:
for v in fsm.results['runlogdetail']:
    if 'Air Injection' in v.__str__():
        if v.msg.severity >= 600:
            print(v)